In [5]:
from enum import Enum
from solarmed_modeling.fsms import MedState, MedVacuumState
from solarmed_modeling.fsms.med import FsmInputs as MedFsmInputs

class MedMode(Enum):
    OFF = 0
    IDLE = 1
    ACTIVE = 2
    
fsm_inputs_table: dict[tuple[MedMode, MedState], MedFsmInputs] = {
    # med_mode = OFF
    (MedMode.OFF, MedState.OFF):               MedFsmInputs(med_active=False, med_vacuum_state=MedVacuumState.OFF),
    (MedMode.OFF, MedState.GENERATING_VACUUM): MedFsmInputs(med_active=False, med_vacuum_state=MedVacuumState.OFF),
    (MedMode.OFF, MedState.IDLE):              MedFsmInputs(med_active=False, med_vacuum_state=MedVacuumState.OFF),
    (MedMode.OFF, MedState.STARTING_UP):       MedFsmInputs(med_active=False, med_vacuum_state=MedVacuumState.OFF),
    (MedMode.OFF, MedState.SHUTTING_DOWN):     MedFsmInputs(med_active=False, med_vacuum_state=MedVacuumState.OFF),
    (MedMode.OFF, MedState.ACTIVE):            MedFsmInputs(med_active=False, med_vacuum_state=MedVacuumState.OFF),
    
    # med_mode = IDLE
    (MedMode.IDLE, MedState.OFF):               MedFsmInputs(med_active=False, med_vacuum_state=MedVacuumState.HIGH),
    (MedMode.IDLE, MedState.GENERATING_VACUUM): MedFsmInputs(med_active=False, med_vacuum_state=MedVacuumState.HIGH),
    (MedMode.IDLE, MedState.IDLE):              MedFsmInputs(med_active=False, med_vacuum_state=MedVacuumState.LOW),
    (MedMode.IDLE, MedState.STARTING_UP):       MedFsmInputs(med_active=False, med_vacuum_state=MedVacuumState.LOW),
    (MedMode.IDLE, MedState.SHUTTING_DOWN):     MedFsmInputs(med_active=False, med_vacuum_state=MedVacuumState.LOW),
    (MedMode.IDLE, MedState.ACTIVE):            MedFsmInputs(med_active=False, med_vacuum_state=MedVacuumState.LOW),
    
    # med_mode = ACTIVE
    (MedMode.ACTIVE, MedState.OFF):               MedFsmInputs(med_active=False, med_vacuum_state=MedVacuumState.HIGH),
    (MedMode.ACTIVE, MedState.GENERATING_VACUUM): MedFsmInputs(med_active=False, med_vacuum_state=MedVacuumState.HIGH),
    (MedMode.ACTIVE, MedState.IDLE):              MedFsmInputs(med_active=True,  med_vacuum_state=MedVacuumState.LOW),
    (MedMode.ACTIVE, MedState.STARTING_UP):       MedFsmInputs(med_active=True,  med_vacuum_state=MedVacuumState.LOW),
    (MedMode.ACTIVE, MedState.SHUTTING_DOWN):     MedFsmInputs(med_active=False, med_vacuum_state=MedVacuumState.LOW),
    (MedMode.ACTIVE, MedState.ACTIVE):            MedFsmInputs(med_active=True,  med_vacuum_state=MedVacuumState.LOW),
}

def get_valid_modes(fsm_inputs: MedFsmInputs) -> list[MedMode]:
    """ Return valid modes given a set of fsm inputs that can be applied to the system (generated by studying the FSM possible evolutions) """
    valid_modes = [
        key[0] for key, value in fsm_inputs_table.items() if value == fsm_inputs
    ]
    return valid_modes

# Find suitable FsmInputs given a pair of desired operating mode and current state
print(f"{fsm_inputs_table[ (MedMode.ACTIVE, MedState.IDLE) ]=}")

# Given a valid set of FsmInputs, find the possible operating modes
get_valid_modes( MedFsmInputs(med_active=False, med_vacuum_state=MedVacuumState.LOW) )


fsm_inputs_table[ (MedMode.ACTIVE, MedState.IDLE) ]=FsmInputs(med_active=True, med_vacuum_state=<MedVacuumState.LOW: 1>)


[<MedMode.IDLE: 1>,
 <MedMode.IDLE: 1>,
 <MedMode.IDLE: 1>,
 <MedMode.IDLE: 1>,
 <MedMode.ACTIVE: 2>]

In [3]:
MedFsmInputs(med_active=0, med_vacuum_state=0)


FsmInputs(med_active=False, med_vacuum_state=<MedVacuumState.OFF: 0>)

In [7]:
from enum import Enum

class FsmsTOptimVarIdsMapping(Enum):
    """
    Mapping between optimization integer decision variable ids and finite state
    machines ones. 
    Using an Enum allows for bi-directional lookups compared to a dictionary
    
    Structure:
    optim_var_id = fsm_var_id
    
    Examples:
    # Convert from optim id to fsm id
    print(f"optim_id: sf_active -> fsm id: {VarIdsOptimToFsmsMapping.sf_active.value}")

    # Convert from fsm id to optim id
    print(f"fsm id: qsf -> optim_id: {VarIdsOptimToFsmsMapping('qsf').name}")
    """
    sf_active = "sf_active"
    ts_active = "ts_active"
    med_active = "med_mode"
    med_vacuum_state = "med_mode"

FsmsTOptimVarIdsMapping["med_vacuum_state"]


<FsmsTOptimVarIdsMapping.med_active: 'med_mode'>

In [10]:
from dataclasses import dataclass

@dataclass
class FsmsTOptimVarIdsMapping:
    sf_active: str = "sf_active"
    ts_active: str = "ts_active"
    med_active: str = "med_mode"
    med_vacuum_state: str = "med_mode"

FsmsTOptimVarIdsMapping.med_vacuum_state
getattr(FsmsTOptimVarIdsMapping, "med_vacuum_state")


'med_mode'

In [19]:
class OptimToFsmsVarIdsMapping:
    sf_active = "sf_active"
    ts_active = "ts_active"
    med_mode = ("med_active", "med_vacuum_state")
    
OptimToFsmsVarIdsMapping.med_mode
getattr(OptimToFsmsVarIdsMapping, "med_mode")


['med_active', 'med_vacuum_state']

In [4]:
import itertools

# Example input: A list containing N lists with the same number of elements
lists = [
    [1, 2],
    [2, 3],
    [3, 4]
]

# Generate all combinations
combinations = list(itertools.product(*lists))

# # Filter combinations to ensure elements are distinct
# unique_combinations = [combo for combo in combinations if len(set(combo)) == len(combo)]

# # Print the results
print("filtering")
for combo in list(set(combinations)):
    print(combo)
    
print("without filtering")
for combo in combinations:
    print(combo)


filtering
(2, 2, 3)
(2, 3, 3)
(1, 2, 3)
(1, 3, 4)
(2, 2, 4)
(1, 3, 3)
(2, 3, 4)
(1, 2, 4)
without filtering
(1, 2, 3)
(1, 2, 4)
(1, 3, 3)
(1, 3, 4)
(2, 2, 3)
(2, 2, 4)
(2, 3, 3)
(2, 3, 4)


In [ ]:
problem_instance = ""


In [ ]:
from typing import NamedTuple
import numpy as np
import itertools
from dataclasses import asdict

from solarmed_optimization import med_fsm_inputs_table
from solarmed_optimization.utils import get_valid_modes


class OptimToFsmsVarIdsMapping(NamedTuple):
    sf_active: tuple = ("sf_active",)
    ts_active: tuple = ("ts_active",)
    med_mode: tuple  = ("med_active", "med_vacuum_state")
    
    
# To simplify, use a list of arrays with shape (n_updates(i), ) to build bounds, later reshape it to (sum(n_updates(i)))
# Initialization
integer_dec_vars_mapping: dict[str, np.ndarray[list[int]]] = {
    var_id: np.full((getattr(problem_instance.dec_var_updates, var_id), ), list[int], dtype=object) for var_id in problem_instance.dec_var_int_ids
}
# Lower box-bounds
lbb: list[np.ndarray[float | int]] = [
    np.full((n_updates, ), np.nan, dtype=float) for n_updates in asdict(problem_instance.dec_var_updates).values()
]
# Upper box-bounds
ubb: list[np.ndarray[float | int]] = [
    np.full((n_updates, ), np.nan, dtype=float) for n_updates in asdict(problem_instance.dec_var_updates).values()
]
    
for initial_state, fsm_data, lookup_table in zip([problem_instance.model_dict["sf_ts_state"], problem_instance.model_dict["med_state"]],
                                                 [problem_instance.fsm_sfts_data, problem_instance.fsm_med_data],
                                                 [None, med_fsm_inputs_table]):
            
    paths_df = fsm_data.paths_df
    valid_inputs = fsm_data.valid_inputs
    fsm_input_ids: list[str] = fsm_data.metadata["input_ids"] 

    # Extract indexes of possible paths from initial state
    paths_from_state_idxs: np.ndarray = paths_df[paths_df["0"] == initial_state.value].index.to_numpy() # dim: (n paths, )
    # Get valid inputs from initial states using those indexes
    valid_inputs_from_state: np.ndarray = valid_inputs[paths_from_state_idxs] # dim: (n paths, n horizon, n inputs)

    # Get the unique discrete values for each input
    for optim_var_id in problem_instance.dec_var_int_ids:
        n_updates: int = getattr(problem_instance.dec_var_updates, optim_var_id)
        fsm_var_ids: list[str] = getattr(OptimToFsmsVarIdsMapping, optim_var_id)
        input_idx_in_dec_vars: int = problem_instance.dec_var_ids.index(optim_var_id)
        
        for step_idx in range(n_updates):
            # Get unique values for each fsm input at step x
            # discrete_bounds: (n fsm inputs, n unique) [[fsm_input i unique values at step x], ..., [fsm_input N unique values at step x]]
            fsm_discrete_bounds = [np.unique(valid_inputs_from_state[:, step_idx, fsm_input_idx]) for fsm_input_idx in range(len(fsm_var_ids))]
            
            # Translate fsm inputs discrete bounds to optimization variable discrete bounds
            if len(fsm_discrete_bounds) == 1:
                # Optimization variable maps to a single fsm input
                optim_var_discrete_bounds = fsm_discrete_bounds[0]
            else:
                # Optimization variable wraps multiple fsm inputs
                combinations = list(itertools.product(*fsm_discrete_bounds))
                optim_var_values = [get_valid_modes(combo, lookup_table=lookup_table) for combo in combinations]
                
                optim_var_discrete_bounds = np.unique(np.array(*optim_var_values))

            # Update mapping
            integer_dec_vars_mapping[optim_var_id][step_idx] = optim_var_discrete_bounds
            # Update bounds
            ubb[input_idx_in_dec_vars][step_idx] = len(optim_var_discrete_bounds)-1
            lbb[input_idx_in_dec_vars][step_idx] = 0
